<a href="https://colab.research.google.com/github/chogh824/machine-learning/blob/main/5_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 트리의 앙상블


**정형 데이터** : CSV나 데이터베이스, 혹은 엑셀에 저장하기 쉬운 데이터. 어떤 구조로 되어 있다는 뜻. **이를 다루는데 가장 뛰어난 성과를 내는 알고리즘이 앙상블 학습**

**비정형 데이터** : 데이터베이스나 엑셀로 표현하기 어려운 것들. 텍스트 데이터, 디지털 카메라로 찍은 사진, 핸드폰으로 듣는 디지털 음악 등이 있음. **이를 가장 잘 다루는 알고리즘은 신경망 알고리즘!**

## 랜덤포레스트

앙상블 학습의 대표 주자 중 하나, 안정적인 성능 덕에 널리 사용됨

결정 트리를 랜덤하게 만들어 결정 트리(나무)의 숲을 만듬. 그리고 각 결정 트리의 예측을 사용해 최종 예측을 만듬.

각 트리를 훈련하기 위한 데이터를 랜덤하게 만드는데, 우리가 입력한 훈련 데이터에서 랜덤하게 샘플을 추출하여 훈련 데이터를 만듬. 한 샘플이 중복되어 추출될 수도 있음!


---

**부트스트랩 샘플:** 일반적으로 훈련 세트의 크기와 같음.

각 노드를 분할할 때 전체 특성 중에서 일부 특성을 무작위로 고른 다음 이 중에서 최선의 분할을 찾음. 분류 모델인 RandomForestClassifier는 기본적으로 전체 특성 개수의 제곱근만큼의 특성을 선택함. 이는 자체적으로 모델을 평가하는 점수를 얻을 수도 있음.

즉 4개의 특성이 있다면 노드마다 2개를 랜덤하게 선택하여 사용함. 다만 회귀모델인 RandomForestRegressor는 전체 특성을 사용함!

사이킷런의 랜덤 포레스트는 기본적으로 100개의 결정 트리를 이런 방식으로 훈련함.

분류일 때는 각 트리의 클래스별 확률을 평균하여 가장 높은 확률을 가진 클래스를 예측으로 삼음!

회귀일 때는 단순히 각 트리의 예측을 평균함.

랜덤 포레스트는 랜덤하게 선택한 샘플과 특성을 사용하기 때문에 훈련 세트에 과대적합되는 것을 막아주고 검증 세트와 테스트 세트에서 안정적인 성능을 얻을 수 있음. 종종 기본 매개변수 설정만으로도 아주 좋은 결과를 냄!

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))  #과대적합

0.9973541965122431 0.8905151032797809


랜덤 포레스트는 특성 중요도를 계산함! 랜덤 포레스트의 특성 중요도 - 각 결정 트리의 특성 중요도를 취합한 것

In [ ]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


5-1 '결정 트리'에서 만든 특성 중요도와 비교해 보았을 때 랜덤 포레스트로 만든 특성 중요도에서는 당도의 중요도가 감소하고 알코올 도수와 pH 특성의 중요도가 조금 상승한 것을 확인할 수 있음.

특성의 일부를 랜덤하게 선택하여 특성 트리를 훈련했기 때문! ->

하나의 특성에 과도하게 집중하지 않고 좀 더 많은 특성이 훈련에 기여할 기회를 얻음. 이는 과대적합을 줄이고 일반화 성능을 높이는 데 도움이 됨!

부트스트랩은 랜덤으로 훈련 세트에서 샘플을 만들어 내는 것. 부트스트랩 샘플에 포함되지 않고 남는 샘플들은 OOB(Out Of Bag)샘플이라 하며, 이 남는 샘플을 사용하여 부트스트랩 샘플로 훈련한 결정 트리를 평가할 수 있음. 일종의 검증 세트 역할을 하는 것!

In [ ]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42) #True로 해야 OOB 점수를 평균하여 출력해줌

rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


## 엑스트라트리

랜덤 포레스트와 비슷하게 동작. 다만 랜덤하게 노드를 분할하는 것 때문에 계산 속도가 빠름.

100개의 결정 트리를 훈련. 다만 부트스트랩 샘플을 사용하지 않음. 즉 각 결정 트리를 만들 때 전체 훈련 세트를 사용함!

대신 노드를 분할할 때 가장 좋은 분할을 찾는 것이 아니라 무작위로 분할함.

엑스트라 트리가 사용하는 결정 트리가 바로 splitter='random'인 결정트리임

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [ ]:
et.fit(train_input, train_target)
print(et.feature_importances_)  #랜덤 포레스트와 마찬가지로 특성 중요도를 제공함

[0.20183568 0.52242907 0.27573525]


## 그레이디언트 부스팅

깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식

사이킷런에서는 기본적으로 깊이가 3인 결정 트리 100개를 사용

깊이가 얕은 결정 트리를 사용하므로 과대적합에 강하고 일반적으로 높은 일반화 성능을 기대할 수 있음

---

그레이디언트란 이름이 붙은 이유는 경사 하강법을 사용하여 트리를 앙상블에 추가하기 때문

분류에서는 로지스틱 손실 함수를 사용

회귀에서는 평균 제곱 오차 함수(MSE)를 사용함

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [ ]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [ ]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)  # 그레이디언트 부스팅이 랜덤 포레스트보다 당도에 더 집중함

[0.15872278 0.68010884 0.16116839]


트리 훈련에 사용할 훈련 세트의 비율을 정하는 subsample이라는 매개변수의 기본값은 1.0으로, 전체 훈련 세트를 사용한다는 말임. subsample < 1 : 훈련 세트의 일부를 사용.

그레이디언트 부스팅이 랜덤 포레스트보다 조금 더 높은 성능을 얻을 수 있으나, 순서대로 트리를 추가하는 것 때문에 훈련 속도는 느림. n_jobs 매개변수가 없음!

## 히스토그램 기반 그레이디언트 부스팅

**정형 데이터를 다루는 머신러닝 알고리즘 중에 가장 인기가 높은 알고리즘**

먼저 입력 특성을 256개의 구간으로 나눔. 따라서 노드를 분할할 때 최적의 분할을 매우 빠르게 찾을 수 있음!

256개의 구간 중에서 하나를 떼어놓고 누락된 값을 위해서 사용함. 따라서 입력에 누락된 특성이 있더도 이를 따로 전처리할 필요가 없음!

In [ ]:
# 사이킷런 1.0 버전 아래에서는 다음 라인의 주석을 해제하고 실행하세요.
# from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [ ]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


특성 중요도를 계산하기 위해 permutation_importance() 함수를 사용함. 이 함수는 특성을 하나씩 랜덤하게 섞어서 모델의 성능이 변화하는지를 관찰하여 어떤 특성이 중요한지를 계산함. 훈련 세트뿐만 아니라 테스트 세트에도 적용할 수 있고 사이킷런에서 제공하는 추정기 모델에 모두 사용 가능!

In [ ]:
result = permutation_importance(hgb, test_input, test_target, n_repeats=10, #n_repeats 매개변수는 랜덤하게 섞을 횟수를 지정, 기본값은 5
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [ ]:
hgb.score(test_input, test_target)

0.8723076923076923

## XGBoost

사이킷런 말고도 히스토그램 기반 그레이디언트 부스팅 알고리즘을 구현한 라이브러리

In [ ]:
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)  #tree_method='hist' = 히스토그램 기반 그레이디언트 부스팅을 사용할 수 있게 됨!
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9555033709953124 0.8799326275264677


## LightGBM

마이크로소프트에서 만든 히스토그램 기반 그레이디언트 부스팅 라이브러리

In [ ]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884
